Make sure you have the clips in data folder. They should be called:
- `1.MOV`
- `2.MOV`
- `3.MOV`
- `MEDIUM_1.MOV`
- `MEDIUM_2.MOV`
- `MEDIUM_3.MOV`
- `HARD_1.MOV`
- `HARD_2.MOV`
- `HARD_3.MOV`
- `test.mp4`

In [1]:
clips = [
    '1.MOV',
    '2.MOV',
    '3.MOV',
    'MEDIUM_1.MOV',
    'MEDIUM_2.MOV',
    'MEDIUM_3.MOV',
    'HARD_1.MOV',
    'HARD_2.MOV',
    'HARD_3.MOV',
    'test.mp4',
]

clips = [f'data/{clip}' for clip in clips]

In [2]:
import numpy as np
import cv2
import PIL

In [3]:
def imshow(a):
    a = a.clip(0, 255).astype("uint8")
    if a.ndim == 3:
        if a.shape[2] == 4:
            a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
        else:
            a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
    display(PIL.Image.fromarray(a))

In [4]:
def load_video(path):
    cap = cv2.VideoCapture(path)
    if cap.isOpened():
        print('Opened the file successfully.')

    print(f"File name: {path}")
    # Height and width of the video
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)

    print(f"Height: {height}, Width: {width}")

    # Number of frames in the video
    FPS = cap.get(cv2.CAP_PROP_FPS)
    print(f"FPS: {FPS}")

    return cap, int(height), int(width), FPS

In [5]:
test_video, test_height, test_width, test_FPS = load_video(clips[-1])

Opened the file successfully.
File name: data/test.mp4
Height: 1280.0, Width: 720.0
FPS: 25.0


In [6]:
first_clip_foreground = cv2.VideoWriter(
    "test_clip_foreground.avi",
    cv2.VideoWriter_fourcc(*"HEVC"),
    test_FPS,
    (int(test_width), int(test_height)),
    0,
)

foreground_knn = cv2.createBackgroundSubtractorKNN(detectShadows=False)

test_video.set(cv2.CAP_PROP_POS_FRAMES, 0)

while test_video.isOpened():
    ret, frame = test_video.read()
    if not ret:
        break
    first_clip_foreground.write(foreground_knn.apply(frame))

first_clip_foreground.release()

In [7]:
!ffmpeg -hide_banner -loglevel error -i test_clip_foreground.avi -y test_clip_foreground.mp4

In [8]:
# close test video
test_video.release()

In [9]:
test_video, test_height, test_width, test_FPS = load_video(clips[1])

Opened the file successfully.
File name: data/2.MOV
Height: 1920.0, Width: 1080.0
FPS: 29.97002997002997


In [10]:
def proper_opening(img, struct):
  transformed = cv2.morphologyEx(
      cv2.morphologyEx(
          cv2.morphologyEx(img, cv2.MORPH_CLOSE, struct), cv2.MORPH_OPEN, struct
      ), cv2.MORPH_CLOSE, struct
  )

  return np.min(np.stack((transformed, img), axis=2), axis=2)

In [11]:
import cv2
import numpy as np

input_file_path = clips[1]  # Replace "your_input_video.mov" with your input video file path
output_file_path = "test_red_clip.avi"

test_video = cv2.VideoCapture(input_file_path)

if not test_video.isOpened():
    print("Error: Could not open the video file.")
    exit()

# Get video properties
test_FPS = int(test_video.get(cv2.CAP_PROP_FPS))
test_width = int(test_video.get(cv2.CAP_PROP_FRAME_WIDTH))
test_height = int(test_video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*"DIVX")
test_red_clip = cv2.VideoWriter(output_file_path, fourcc, test_FPS, (test_width, test_height))

frame_number = 0
while test_video.isOpened():
    ret, frame = test_video.read()
    frame_number += 1
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        print(frame_number)
        break  # Exit the loop when no frames are retrieved
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Range of red color in HSV
    lower_red = np.array([0, 100, 100])
    upper_red = np.array([10, 255, 255])
    lower_blue = np.array([110, 50, 50])
    upper_blue = np.array([130, 255, 255])

    # Threshold the HSV image to get only red or blue colors
    mask_red = cv2.inRange(hsv, lower_red, upper_red)
    mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)

    # Combine masks
    mask = cv2.bitwise_or(mask_red, mask_blue)
    
    # apply proper opening 
    mask = proper_opening(mask, np.ones((11, 11), np.uint8))
    # dilate mask to fill in holes
    mask = cv2.dilate(mask, np.ones((7, 7), np.uint8), iterations=3)


    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(frame, frame, mask=mask)

    test_red_clip.write(res)

test_video.release()  # Release the video capture
test_red_clip.release()


In [ ]:
!ffmpeg -hide_banner -loglevel error -i test_red_clip.avi -y test_red_clip.mp4

In [ ]:
import cv2
import numpy as np

input_file_path = clips[2]
output_file_path = "test_red_clip.avi"

test_video, test_height, test_width, test_FPS = load_video(input_file_path)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*"DIVX")
test_red_clip = cv2.VideoWriter(
    output_file_path, 
    fourcc, 
    test_FPS, 
    (test_width, test_height),
    )

frame_number = 0
while test_video.isOpened():
    ret, frame = test_video.read()
    frame_number += 1
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        print(frame_number)
        break  # Exit the loop when no frames are retrieved
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Range of colors in HSV
    lower_red = np.array([0, 100, 100])
    upper_red = np.array([12, 255, 255])
    lower_blue = np.array([98, 50, 50])
    upper_blue = np.array([118, 255, 255])
    lower_black = np.array([0, 0, 0])
    upper_black = np.array([255, 255, 30])

    # Threshold the HSV image to get only predefined colors
    mask_red = cv2.inRange(hsv, lower_red, upper_red)
    mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)
    mask_black = cv2.inRange(hsv, lower_black, upper_black)

    # Combine masks
    mask = cv2.bitwise_or(mask_red, mask_blue)
    mask = cv2.bitwise_or(mask, mask_black)
    
    # apply proper opening 
    mask = proper_opening(mask, np.ones((9, 9), np.uint8))
    # dilate mask to fill in holes
    mask = cv2.dilate(mask, np.ones((3, 3), np.uint8), iterations=3)


    # Find contours and draw bounding boxes around detected elements
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Draw bounding boxes on original frame
    for contour in contours:
        if cv2.contourArea(contour) < 1500 or cv2.contourArea(contour) > 50000:
            continue
            
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 2)
        if np.sum(mask_blue[y:y+h, x:x+w]) > np.sum(mask_red[y:y+h, x:x+w]):
            cv2.putText(frame, 'Team Blue', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
        elif np.sum(mask_blue[y:y+h, x:x+w]) < np.sum(mask_red[y:y+h, x:x+w]):
            cv2.putText(frame, 'Team Red', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        else:
            cv2.putText(frame, 'Assasin', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            

    test_red_clip.write(frame)

test_video.release()  # Release the video capture
test_red_clip.release()

Opened the file successfully.
File name: data/3.MOV
Height: 1920.0, Width: 1080.0
FPS: 29.97002997002997
Can't receive frame (stream end?). Exiting ...
1906


In [ ]:
!ffmpeg -hide_banner -loglevel error -i test_red_clip.avi -y test_clip.mp4

**TODO:**
- Add filter for the hand, remove it from the color mask.
- write separate bounding box for the hand.
- fix HSV values for colors, add grey and black.
- For grey, it possible we'll need to take some pictures of the cards and compare them to the video this way.
- try to concatenate overlapping bounding boxes. [Stack Overflow](https://stackoverflow.com/questions/57258173/opencv-join-contours-when-rectangle-overlaps-another-rect)